# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хранить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [1]:
from string import punctuation
from razdel import sentenize
from nltk.tokenize import sent_tokenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from scipy.sparse import csr_matrix, csc_matrix, lil_matrix
from collections import Counter

In [2]:
dvach = open('2ch_corpus.txt', encoding = 'utf-8').read()
news = open('lenta.txt', encoding = 'utf-8').read()

In [3]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [4]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [5]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [6]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0, len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [267]:
sentences_dvach = sent_tokenize(dvach)
sentences_news = sent_tokenize(news)

In [268]:
test_dvach = sentences_dvach[-10:]
test_news = sentences_news[-10:]

In [269]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sentences_dvach[:-10]]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] for text in sentences_news[:-10]]

In [270]:
unigrams_dvach = Counter()
bigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    bigrams_dvach.update(ngrammer(sentence, n=2))
    trigrams_dvach.update(ngrammer(sentence))

unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence, n=2))
    trigrams_news.update(ngrammer(sentence))

In [271]:
matrix_dvach = lil_matrix((len(bigrams_dvach), len(unigrams_dvach)))

id2word_dvach = list(unigrams_dvach)
word2id_dvach = {word:i for i, word in enumerate(id2word_dvach)}

id2bigram_dvach = list(bigrams_dvach)
bigram2id_dvach = {word:i for i, word in enumerate(id2bigram_dvach)}


for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_dvach[bigram2id_dvach[bigram], word2id_dvach[word3]] = (trigrams_dvach[ngram]/bigrams_dvach[bigram])

matrix_dvach = csr_matrix(matrix_dvach)

In [272]:
matrix_news = lil_matrix((len(bigrams_news), len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}


for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2
    matrix_news[bigram2id_news[bigram], word2id_news[word3]] = (trigrams_news[ngram]/bigrams_news[bigram])

matrix_news = csr_matrix(matrix_news)

In [273]:
def generate(matrix, id2word, bigram2id, id2bigram, n=100, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(list(range(matrix.shape[1])), p=matrix[current_idx].toarray()[0])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = bigram2id['<start> <start>']
            
        else:
            bigram = id2bigram[current_idx]
            trigram = bigram + ' ' + id2word[chosen]
            w1, w2, w3 = trigram.split()
            new_bigram = w2 + ' ' + w3
            chosen = bigram2id[new_bigram]
            
        current_idx = chosen
    
    return ' '.join(text)

In [276]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 

In [277]:
def text_perplexity(phrase):
    
    prob = {'dvach':[], 'news':[]}

    for ngram in ngrammer(['<start>'] + ['<start>'] + normalize(phrase) + ['<end>']):
        word1, word2, word3 = ngram.split()
        bigram = word1 + ' ' + word2
        if bigram in bigrams_dvach and ngram in trigrams_dvach:
            prob['dvach'].append(np.log(trigrams_dvach[ngram]/bigrams_dvach[bigram]))
        else:
            prob['dvach'].append(np.log(0.00001))
    
        if bigram in bigrams_news and ngram in trigrams_news:
            prob['news'].append(np.log(trigrams_news[ngram]/bigrams_news[bigram]))
        else:
            prob['news'].append(np.log(0.00001))
            
    return perplexity(prob['dvach']), perplexity(prob['news'])

### Примеры сгенерированных текстов

In [274]:
print(generate(matrix_news, id2word_news, bigram2id_news, id2bigram_news).replace('<end>', '\n'))

с черепно-мозговой травмой полонский в тяжелом состоянии госпитализирована беременная женщина 
 об этом сегодня на всероссийском совещании по проблемам науки заявил в среду постоянный представитель израиля при оон иегуда ланкри сказал что эта программа 
 по оценке журнала интернет и другие комплектующие увеличились в рублевом исчислении в 2,504 раза более 540 мирных жителей в тыл врага передает нтв 
 как сообщает wired представители havenco уже заявили о том что по предварительным сведениям сход вагонов произошел не на что уйдет еще несколько недель выделение второго транша кредита заявил заместитель главы мвд михаил ваничкин сообщает риа новости таможенные органы россии откажутся от


In [306]:
print(generate(matrix_dvach, id2word_dvach, bigram2id_dvach, id2bigram_dvach).replace('<end>', '\n'))

что не все целиком 
 там даже скипать нельзя 
 потому что не так как последние будут продавать свои васянские ретекстуры на беспезда 
 не важно на что вы будете делать если слишком жарко — выходите из лавки на узкую ца 
 тут был один случай или это говно собираются сносить 
 по моему опыту да там одни школьнички травят других школьничков как-то по другому что то ещё терпимо 
 и игра на ведроиде 
 тебя по штанине течёт чмоша-хейтер без аргументов 
 значит скачит на мистеческом хую шамана гадальщика 
 какой подход выбирать 
 для меня тян пустое место


### Рассчет перплексии на тестовой выборке из 10 текстов

In [307]:
print('Значения перплексии текстов:')
for text in test_dvach:
    print(text_perplexity(text)[0])

Значения перплексии текстов:
346.04299406789534
1966.613308078042
23703.411851246256
23671.503714242263
61822.15191056624
44788.894907421505
1193.706497445341
19.399890839584437
118.83316294206847
108.6707949062014


In [309]:
print('Значения перплексии текстов:')
for text in test_news:
    print(text_perplexity(text)[1])

Значения перплексии текстов:
9075.506695032758
2221.207237303329
21616.280010688606
4756.515099496046
321.9359397055297
18938.12599022405
3794.061544807548
50443.03421515644
18132.144650748425
23948.160623023487


## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Вместо того, чтобы приписать слову маленькое значение вероятности, можно добавить в словарь фиктивное слово \<UNK\>. Есть 2 способа определить вероятность такого слова:
* если словарь уже построен, все несловарные слова заменяем на \<UNK\> на этапе нормализации и считаем вероятность \<UNK\> как в случае с любым другим ловом
* если словаря нет, создаем словарь, в который включаем слова, имеющие вероятности выше выбранного порога, либо входящие в топ по вероятности (например, можно брать первые 10000 слов); добавляем \<UNK\> в словарь; слова, не вошедшие в словарь, заменяются на \<UNK\>, вероятность которого считается так же, как в случае 1). 

2. Что такое сглаживание (smoothing)?

Бывают случаи, когда слово присутствует в словаре, но в тестовом тексте появляется в неизвестном контексте (например, слово "змей" есть в словаре, но оно никогда не появлялось после слова "тугарин". Если допустить, что в словаре есть все варианты контекстов, то языковая модель присвоит такому случаю нулевую вероятность. Чтобы этого избежать, необходимо оставить долю вероятности для таких событий, "отщепнув" ее от вероятности наиболее частых контекстов. Такое перераспределение вероятностей называется **сглаживанием**.